# Discover Spotify
To use this file, you first need to manually execute the file 'get_token.py' and follow the instructions in the console.

The resulting token was saved in 'token.txt' and is valid for one hour.

If a valid token has been stored, the following code can be executed.

# Authorize

In [ ]:
import spotipy
import json

with open('token.txt', 'r') as f:
    token = f.read()

sp = spotipy.Spotify(auth=token)

In [ ]:
# Get the current user
# (This is needed to create a playlist later on)

USER = sp.current_user()
USER_ID = USER['id']

# Discover

In [ ]:
def print_top_items(top_items, item_type):
    print(f'\n\nYour top {item_type} are:')
    for i, item in enumerate(top_items):
        item_name = item['name']
        print(f'{i+1}. {item_name}')

In [ ]:
def save_dict_to_json(d, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)

### Top artists and tracks

In [ ]:
# Get the user's top artists
top_artists = sp.current_user_top_artists(limit=50, time_range='medium_term')

# Get the user's top tracks
top_tracks = sp.current_user_top_tracks(limit=50, time_range='medium_term')

# Get the user's top genres
top_genres = top_artists['items'][0]['genres']

In [ ]:
print_top_items(top_artists['items'][0:10], 'artists')
print_top_items(top_tracks['items'][0:10], 'tracks')

save_dict_to_json(top_artists, 'top_artists.json')
save_dict_to_json(top_tracks, 'top_tracks.json')

### Recommendations

Get recommendations based on the user's top artists, genres, and tracks

In [ ]:
seed_artists = [artist['id'] for artist in top_artists['items']][0:2]
seed_genres = []
for artist in top_artists['items']:
    for genre in artist['genres']:
        if genre not in seed_genres:
            seed_genres.append(genre)
seed_genres = seed_genres[0:2]
seed_tracks = [track['id'] for track in top_tracks['items']][0:1]

print(seed_artists)
print(seed_genres)
print(seed_tracks)

recommendations = sp.recommendations(seed_artists=seed_artists, seed_genres=seed_genres, seed_tracks=seed_tracks, limit=100)

Save recommendations to a new playlist

In [ ]:
playlist = sp.user_playlist_create(user=USER_ID, name='Test', public=False, collaborative=False, description='Test', )
sp.user_playlist_add_tracks(user=USER_ID, playlist_id=playlist['id'], tracks=[track['uri'] for track in recommendations['tracks']])